In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")

In [5]:
import csv
import datetime
import os
from collections import defaultdict

Create a function to load reviews and movies using lambda functions to parse specific fields in datasets.

In [11]:
def parse_date(r, k):
    try:
        return datetime.strptime(r[k], "%d-%b-%Y") 
    except:
        return None

In [16]:
def load_reviews(path, **kwargs):
    options = {"fieldnames": ("userid", "movieid", "rating", "timestamp"), "delimiter":"\t"}
    options.update(kwargs)
    
    parse_date = lambda r, k: datetime.datetime.fromtimestamp(float(r[k]))
    parse_int = lambda r, k: int(r[k])
    
    with open(path, "rb") as reviews:
        reader = csv.DictReader(reviews, **options)
        for row in reader:
            row["userid"] = parse_int(row, "userid")
            row["movieid"] = parse_int(row, "movieid")
            row["rating"] = parse_int(row, "rating")
            row["timestamp"] = parse_date(row, "timestamp")
            yield row

In [14]:
def load_movies(path, **kwargs):
    options = {"fieldnames":("movieid", "title", "release", "video", "url"), "delimiter":"|", "restkey":"genre"}
    options.update(kwargs)
    
    parse_int = lambda r, k: int(r[k])
    
    with open(path, "rb") as movies:
        reader = csv.DictReader(movies, **options)
        for row in reader:
            row["movieid"] = parse_int(row, "movieid")
            row["release"] = parse_date(row, "release")
            row["video"] = parse_date(row, "video")
            yield row

Create a MovieLens class that will be used as a data structure to build the recommender model.

In [42]:
import heapq
from operator import itemgetter
from math import sqrt

In [84]:
class MovieLens(object):
    """Date structure for recommendation model."""
    
    def __init__(self, udata, uitem):
        self.udata = udata
        self.uitem = uitem
        self.movies = {}
        self.reviews = defaultdict(dict)
        self.load_dataset()
        
    def load_dataset(self):
        for movie in load_movies(self.uitem):
            self.movies[movie["movieid"]] = movie
            
        for review in load_reviews(self.udata):
            self.reviews[review["userid"]][review["movieid"]] = review
    
    def reviews_for_movie(self, movieid):
        """Yields reviews for a given movie."""
        for review in self.reviews.values():
            if movieid in review:
                yield review[movieid]
    
    def average_reviews(self):
        """Average star ratings for all movies."""
        for movieid in self.movies:
            reviews = list(r["rating"] for r in self.reviews_for_movie(movieid))
            average = sum(reviews) / float(len(reviews))
            yield (movieid, average, len(reviews))
    
    def bayesian_average(self, c=59, m=3):
        """Reports Bayesian average to penalize movies with small number of reviews."""
        for movieid in self.movies:
            reviews = list(r["rating"] for r in self.reviews_for_movie(movieid))
            average = ((c*m) + sum(reviews)) / float(c + len(reviews))
            yield(movieid, average, len(reviews))
        
    def top_rated(self, n = 10, C=59, M=3):
        """Yields top n rated movies."""
        return heapq.nlargest(n, self.bayesian_average(c=C, m=M), key=itemgetter(1))
    
    def shared_preferences(self, criticA, criticB):
        """Returns intersection of ratings for 2 critics."""
        if criticA not in self.reviews:
            raise KeyError("Couldn't find critic %s in data" % criticA)
        if criticB not in self.reviews:
            raise KeyError("Couldn't find critic %s in data" % criticB)
        
        moviesA = set(self.reviews[criticA].keys())
        moviesB = set(self.reviews[criticB].keys())
        shared = moviesA & moviesB
        
        #create a shared reviews dictionary to return
        reviews = {}
        for movieid in shared:
            reviews[movieid] = (self.reviews[criticA][movieid]["rating"], self.reviews[criticB][movieid]["rating"])
        return reviews
    
    def shared_critics(self, movieA, movieB):
        """Returns intersection of critics for A and B."""
        if movieA not in self.movies:
            raise KeyError("Couldn't find movie '%s' in data." % movieA)
        if movieB not in self.movies:
            raise KeyError("Couldn't find movie '%s' in data." % movieB)
        
        criticsA = set(critic for critic in self.reviews if movieA in self.reviews[critic])
        criticsB = set(critic for critic in self.reviews if movieB in self.reviews[critic])
        shared = criticsA & criticsB
        
        reviews = {}
        for critic in shared:
            reviews[critic] = (self.reviews[critic][movieA]["rating"], self.reviews[critic][movieB]["rating"])
        return reviews
    
    def euclidean_distance(self, criticA, criticB):
        """Report Euclidean distance of two critics on shared preference vectors."""
        preferences = self.shared_preferences(criticA, criticB)
        if len(preferences) == 0: return 0 #if no movies rated in common
        sumSquares = sum([pow(a -b, 2) for a, b in preferences.values()])
        return 1.0/(1.0 + sqrt(sumSquares))
    
    def manhattan_distance(self, criticA, criticB):
        """Report Manhattan distance of two critics on shared preference vectors."""
        preferences = self.shared_preferences(criticA, criticB)
        if len(preferences) == 0: return 0
        manhattan = sum([abs(a - b) for a, b in preferences.values()])
        return 1.0 / (1.0 + manhattan)
    
    def pearson_correlation(self, criticA, criticB):
        """Return Pearson correlation with 2 critics on shared vectors."""
        preferences = self.shared_preferences(criticA, criticB)
        length = len(preferences)
        if length == 0: return 0
        sumA = sumB = sumSquareA = sumSquareB = sumProducts = 0
        for a, b in preferences.values():
            sumA += a
            sumB += b
            sumSquareA += pow(a, 2)
            sumSquareB += pow(b, 2)
            sumProducts += a * b
        numerator = (sumProducts*length) - sumA * sumB
        denominator = sqrt(((sumSquareA*length) - pow(sumA, 2))*((sumSquareB*length) - pow(sumB, 2)))
        if denominator == 0: return 0
        return abs(numerator / denominator)
    
    def similar_critics(self, user, metric="euclidean", n=None):
        """Finds and ranks similar critics for the user according to the specified distance metric."""
        metrics = {"euclidean":self.euclidean_distance, "pearson": self.pearson_correlation, "manhattan":self.manhattan_distance}
        distance = metrics.get(metric, None)
        
        #error handling
        if user not in self.reviews:
            raise KeyError("Unknown user, '%s'." % user)
        if not distance or not callable(distance):
            raise KeyError("Unknown distance metric %s." % metric)
        critics = {}
        for critic in self.reviews:
            if critic == user:
                continue
            critics[critic] = distance(user, critic)
        if n: return heapq.nlargest(n, critics.items(), key=itemgetter(1))
        return critics
    
    def similar_items(self, movie, metric="euclidean", n = None):
        metrics = {"euclidean":self.euclidean_distance, "pearson": self.pearson_correlation, "manhattan":self.manhattan_distance}
        distance = metrics.get(metric, None)
        
        if movie not in self.reviews:
            raise KeyError("Unknown movie, '%s'." % movie)
        if not distance or not callable(distance):
            raise KeyError("Unknown distance metric '%s'." % metric)
        
        items = {}
        for item in self.movies:
            if item == movie: continue
            items[item] = distance(item, movie)
        if n: return heapq.nlargest(n, items.items(), key= itemgetter(1))
        return items
    
    def predict_ranking(self, user, movie, metric="euclidean", critics=None):
        """Predicts ranking user might give a movie based on weighted verage of similar critics."""
        critics = critics or self.similar_critics(user, metric=metric)
        total = 0.0
        simsum = 0.0
        for critic, similarity in critics.items():
            if movie in self.reviews[critic]:
                total += similarity*self.reviews[critic][movie]["rating"]
                simsum += similarity
        if simsum == 0.0: return 0.0
        return total / simsum
    
    def predict_all_rankings(self, user, metric="euclidean", n = None):
        """Predicts all rankings for all movies, if n specified return top n with predictions."""
        critics = self.similar_critics(user, metric=metric)
        movies = {movie:self.predict_ranking(user, movie, metric, critics) for movie in self.movies}
        if n: return heapq.nlargest(n, movies.items(), key=itemgetter(1))
        return movies

In [88]:
data = "u.data"
item = "u.item"
model = MovieLens(data, item)

In [39]:
for mid, avg, num in model.top_rated(10, C=25, M=3.52986):
    title = model.movies[mid]["title"]
    print "[%0.3f average rating (%i reviews)] %s" %(avg, num, title) 

[4.394 average rating (298 reviews)] Schindler's List (1993)
[4.371 average rating (283 reviews)] Shawshank Redemption, The (1994)
[4.370 average rating (243 reviews)] Casablanca (1942)
[4.324 average rating (583 reviews)] Star Wars (1977)
[4.316 average rating (112 reviews)] Close Shave, A (1995)
[4.312 average rating (267 reviews)] Usual Suspects, The (1995)
[4.302 average rating (118 reviews)] Wrong Trousers, The (1993)
[4.296 average rating (209 reviews)] Rear Window (1954)
[4.244 average rating (390 reviews)] Silence of the Lambs, The (1991)
[4.240 average rating (413 reviews)] Godfather, The (1972)


In [25]:
print "Average number of reviews per movie:", float(sum(num for mid, avg, num in model.average_reviews())) / len(model.movies)

Average number of reviews per movie: 59.4530321046


In [31]:
print "Average reviw rating:", float(sum(avg*num for mid, avg, num in model.average_reviews())) \
/float(sum(num for mid, avg, num in model.average_reviews()))

Average reviw rating: 3.52986


In [45]:
print model.euclidean_distance(232, 532)

0.102302162992


In [48]:
print model.manhattan_distance(232, 532)

0.0227272727273


In [54]:
print model.pearson_correlation(232, 532)

0.0602579353839


Find most similar users for user 232.

In [89]:
for item in model.similar_critics(232, "euclidean", n = 10):
    print "%4i: %0.4f" % item

 688: 1.0000
 914: 1.0000
  47: 0.5000
  78: 0.5000
 170: 0.5000
 335: 0.5000
 341: 0.5000
 101: 0.4142
 155: 0.4142
 309: 0.4142


In [90]:
for item in model.similar_critics(232, "pearson", n = 10):
    print "%4i: %0.4f" % item

  33: 1.0000
  36: 1.0000
 155: 1.0000
 260: 1.0000
 289: 1.0000
 302: 1.0000
 309: 1.0000
 317: 1.0000
 511: 1.0000
 769: 1.0000


In [69]:
print model.predict_ranking(422, 50)
print model.predict_ranking(422, 50, "pearson")

4.35413151722
4.3566797826


In [91]:
for mid, rating in model.predict_all_rankings(578, "pearson", 10):
    print "%0.4f: %s" % (rating, model.movies[mid]["title"])

5.0000: Prefontaine (1997)
5.0000: Santa with Muscles (1996)
5.0000: Marlene Dietrich: Shadow and Light (1996) 
5.0000: Star Kid (1997)
5.0000: Aiqing wansui (1994)
5.0000: Someone Else's America (1995)
5.0000: Great Day in Harlem, A (1994)
5.0000: Saint of Fort Washington, The (1993)
4.9539: Anna (1996)
4.8175: Innocents, The (1961)


In [95]:
import pickle

In [96]:
class Recommender(object):

    @classmethod
    def load(klass, pickle_path):
        """
        Instantiates the class by deserializing the pickle. Note that the
        object returned may not be an exact match to the code in this
        class (if it was saved before updates).
        """
        with open(pickle_path, 'rb') as pkl:
            return pickle.load(pkl)

    def __init__(self, udata, description=None):
        self.udata   = udata
        self.users   = None
        self.movies  = None
        self.reviews = None

        # Descriptive properties
        self.build_start  = None
        self.build_finish = None
        self.description  = None

        # Model properties
        self.model        = None
        self.features     = 2
        self.steps        = 5000
        self.alpha        = 0.0002
        self.beta         = 0.02

        self.load_dataset()

    def load_dataset(self):
        """
        Loads an index of users and movies as a heap and a reviews table
        as a N x M array where N is the number of users and M is the number
        of movies. Note that order matters so that we can look up values
        outside of the matrix!
        """
        self.users  = set([])
        self.movies = set([])
        for review in load_reviews(self.udata):
            self.users.add(review['userid'])
            self.movies.add(review['movieid'])

        self.users  = sorted(self.users)
        self.movies = sorted(self.movies)

        self.reviews = np.zeros(shape=(len(self.users), len(self.movies)))
        for review in load_reviews(self.udata):
            uid = self.users.index(review['userid'])
            mid = self.movies.index(review['movieid'])
            self.reviews[uid, mid] = review['rating']

    def build(self, output=None, alternate=False):
        """
        Trains the model by employing matrix factorization on our training
        data set, the sparse reviews matrix. The model is the dot product
        of the P and Q decomposed matrices from the factorization.
        """
        options = {
            'K':     self.features,
            'steps': self.steps,
            'alpha': self.alpha,
            'beta':  self.beta,
        }

        self.build_start = time.time()
        nnmf = factor2 if alternate else factor
        self.P, self.Q = nnmf(self.reviews, **options)
        self.model = np.dot(self.P, self.Q.T)
        self.build_finish = time.time()

        if output:
            self.dump(output)

    def dump(self, pickle_path):
        """
        Dump the object into a serialized file using the pickle module.
        This will allow us to quickly reload our model in the future.
        """
        with open(pickle_path, 'wb') as pkl:
            pickle.dump(self, pkl)

    def sparsity(self):
        """
        Report the percent of elements that are zero in the array
        """
        return 1 - self.density()

    def density(self):
        """
        Return the percent of elements that are nonzero in the array
        """
        nonzero = float(np.count_nonzero(self.reviews))
        return nonzero / self.reviews.size

    def error_rate(self):
        """
        Compute the sum squared error of the trained model.
        """
        error = 0.0
        rows, cols = self.reviews.shape
        for idx in xrange(rows):
            for jdx in xrange(cols):
                if self.reviews[idx, jdx] > 0:
                    error += (self.model[idx, jdx] - self.reviews[idx, jdx]) ** 2
                    print error
        return error

    def predict_ranking(self, user, movie):
        uidx = self.users.index(user)
        midx = self.movies.index(movie)
        return self.model[uidx, midx]

    def top_rated(self, user, n=12):
        movies = [(mid, self.predict_ranking(user, mid)) for mid in self.movies]
        return heapq.nlargest(n, movies, key=itemgetter(1))

In [97]:
model = Recommender(data)

In [99]:
print "Model sparsity:", model.sparsity()
print "Model density:", model.density()

Model sparsity: 0.936953306358
Model density: 0.0630466936422


In [100]:
def initialize(R, K):
    N, M = R.shape
    P = np.random.randn(N, K)
    Q = np.random.randn(M, K)
    return P, Q

In [101]:
def factor(R, P=None, Q=None, K=2, steps=5000, alpha=0.0002, beta=0.02):
    """
    Performs matrix factorization on R with given parameters.
    :param R: A matrix to be factorized, dimension N x M
    :param P: an initial matrix of dimension N x K
    :param Q: an initial matrix of dimension M x K
    :param K: the number of latent features
    :param steps: the maximum number of iterations to optimize in
    :param alpha: the learning rate for gradient descent
    :param beta:  the regularization parameter
    :returns: final matrices P and Q
    """

    if not P or not Q:
        P, Q = initialize(R, K)
    Q = Q.T

    rows, cols = R.shape
    for step in xrange(steps):
        for i in xrange(rows):
            for j in xrange(cols):
                if R[i,j] > 0:
                    eij = R[i,j] - np.dot(P[i,:], Q[:,j])
                    for k in xrange(K):
                        P[i,k] = P[i,k] + alpha * (2 * eij * Q[k,j] - beta * P[i,k])
                        Q[k,j] = Q[k,j] + alpha * (2 * eij * P[i,k] - beta * Q[k,j])

        e  = 0
        for i in xrange(rows):
            for j in xrange(cols):
                if R[i,j] > 0:
                    e = e + pow(R[i,j] - np.dot(P[i,:], Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * (pow(P[i,k], 2) + pow(Q[k,j], 2))
        if e < 0.001:
            break

    return P, Q.T